<a href="https://colab.research.google.com/github/aditj/IRS-Paper/blob/master/IRS_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import math

In [ ]:
M = 2

In [ ]:
beta = 1
mean = 0
Num_Taps = 20
Num_channels = 1
H_d = mean + (math.sqrt(beta) * np.random.randn(Num_Taps, Num_channels*2).view(np.complex128))

In [ ]:
from scipy.stats import rice

In [ ]:
def rician(beta,k,h_d ,rows,cols):
  det = math.sqrt(beta/(1+k))*h_d
  ran = math.sqrt(beta*k/((1+k)*2)) * np.random.randn(rows, cols*2).view(np.complex128))
  return det + ran

In [ ]:
beta = 1
mean = 2+6j
Num_Taps = 20
Num_channels = 64
H_n = mean + (math.sqrt(beta) * np.random.randn(Num_Taps, Num_channels*2).view(np.complex128))

In [ ]:
H_d.shape, H_n.shape

((20, 1), (20, 64))

In [ ]:
H = np.concatenate((H_d,H_n), axis = 1)

In [ ]:
IRS_config = np.exp(2j*np.random.randint(M,size=H_n.shape[1])*np.pi/M)
Direct = np.ones(H_n.shape[1]).astype(np.complex128)
G = np.concatenate((Direct,IRS_config), axis = 1)

In [ ]:
H_f = np.multiply(G,H)